# **ONSAGER TESTING**

## **Installs**

### Firedrake

In [22]:
try:
    !wget "https://fem-on-colab.github.io/releases/firedrake-install-development-real.sh" -O "/tmp/firedrake-install.sh"
    !bash "/tmp/firedrake-install.sh"
    from firedrake import *  # noqa: F401
except:
    from firedrake import *  # noqa: F401

--2026-02-04 16:17:19--  https://fem-on-colab.github.io/releases/firedrake-install-development-real.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.110.153, 185.199.111.153, 185.199.108.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4775 (4.7K) [application/x-sh]
Saving to: ‘/tmp/firedrake-install.sh’

/tmp/firedrake-inst 100%[===================>]   4.66K  --.-KB/s    in 0s      

2026-02-04 16:17:19 (69.8 MB/s) - ‘/tmp/firedrake-install.sh’ saved [4775/4775]

+ INSTALL_PREFIX=/usr/local
++ echo /usr/local
++ awk -F/ '{print NF-1}'
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ FIREDRAKE_INSTALLED=/usr/local/share/fem-on-colab/firedrake.installed
+ [[ ! -f /usr/local/share/fem-on-colab/firedrake.installed ]]
+ set +x
























#########################################################

### Gmsh

In [21]:
try:
    !wget "https://fem-on-colab.github.io/releases/gmsh-install.sh" -O "/tmp/gmsh-install.sh"
    !bash "/tmp/gmsh-install.sh"
    import gmsh  # noqa: F401
except:
    import gmsh  # noqa: F401

--2026-02-04 16:17:13--  https://fem-on-colab.github.io/releases/gmsh-install.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.110.153, 185.199.111.153, 185.199.108.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3497 (3.4K) [application/x-sh]
Saving to: ‘/tmp/gmsh-install.sh’

/tmp/gmsh-install.s 100%[===================>]   3.42K  --.-KB/s    in 0s      

2026-02-04 16:17:13 (42.6 MB/s) - ‘/tmp/gmsh-install.sh’ saved [3497/3497]

+ INSTALL_PREFIX=/usr/local
++ awk -F/ '{print NF-1}'
++ echo /usr/local
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ GMSH_INSTALLED=/usr/local/share/fem-on-colab/gmsh.installed
+ [[ ! -f /usr/local/share/fem-on-colab/gmsh.installed ]]
+ H5PY_INSTALL_SCRIPT_PATH=https://github.com/fem-on-colab/fem-on-colab.github.io/raw/2ffb5740/releases/h5py-install.sh
+ [[ https://gith

### Other

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## **Code**

### Mesh

In [35]:
def rect_with_hole_mesh(r=0.5, left=1.0, right=4.0, updown=1.0, h=0.2):
    '''
    r = hole radius
    left = space to left of hole centre
    right = space to right of hole centre
    updown = space above / below hole centre
    h = (target) mesh size
    '''
    gmsh.initialize()
    gmsh.model.add("rect_with_hole")

    # Derived rectangle bounds
    xmin = -left
    xmax = right
    ymin = -updown
    ymax = updown


    # -------------------
    # Rectangle
    # -------------------
    p1 = gmsh.model.geo.addPoint(xmin, ymin, 0, h)
    p2 = gmsh.model.geo.addPoint(xmax, ymin, 0, h)
    p3 = gmsh.model.geo.addPoint(xmax, ymax, 0, h)
    p4 = gmsh.model.geo.addPoint(xmin, ymax, 0, h)

    l_bottom = gmsh.model.geo.addLine(p1, p2)
    l_right  = gmsh.model.geo.addLine(p2, p3)
    l_top    = gmsh.model.geo.addLine(p3, p4)
    l_left   = gmsh.model.geo.addLine(p4, p1)

    rect_loop = gmsh.model.geo.addCurveLoop(
        [l_bottom, l_right, l_top, l_left]
    )


    # -------------------
    # Circular hole (centred at origin)
    # -------------------
    c  = gmsh.model.geo.addPoint(0, 0, 0, h)
    cp = gmsh.model.geo.addPoint( r, 0, 0, h)
    cn = gmsh.model.geo.addPoint(-r, 0, 0, h)
    ct = gmsh.model.geo.addPoint(0,  r, 0, h)
    cb = gmsh.model.geo.addPoint(0, -r, 0, h)

    a1 = gmsh.model.geo.addCircleArc(cp, c, ct)
    a2 = gmsh.model.geo.addCircleArc(ct, c, cn)
    a3 = gmsh.model.geo.addCircleArc(cn, c, cb)
    a4 = gmsh.model.geo.addCircleArc(cb, c, cp)

    hole_loop = gmsh.model.geo.addCurveLoop([a1, a2, a3, a4])


    # -------------------
    # Surface with hole
    # -------------------
    surface = gmsh.model.geo.addPlaneSurface([rect_loop, hole_loop])

    gmsh.model.geo.synchronize()


    # -------------------
    # Physical groups (THIS is the important bit)
    # -------------------
    gmsh.model.addPhysicalGroup(1, [l_left], tag=1)
    gmsh.model.setPhysicalName(1, 1, "inlet")

    gmsh.model.addPhysicalGroup(1, [l_right], tag=2)
    gmsh.model.setPhysicalName(1, 2, "outlet")

    gmsh.model.addPhysicalGroup(1, [l_top, l_bottom], tag=3)
    gmsh.model.setPhysicalName(1, 3, "wall")

    gmsh.model.addPhysicalGroup(1, [a1, a2, a3, a4], tag=4)
    gmsh.model.setPhysicalName(1, 4, "hole")

    gmsh.model.addPhysicalGroup(2, [surface], tag=5)


    # -------------------
    # Mesh
    # -------------------
    gmsh.model.mesh.generate(2)
    gmsh.write("rect_with_hole.msh")
    gmsh.finalize()

    return Mesh("rect_with_hole.msh")

### Navier–Stokes

In [ ]:
def navier_stokes(h=0.2, degree=2, sigma=10.0, nu=0.01, dt=2**-5, T=2**-3, download=False):
    # ---------
    #   Setup
    # ---------
    # Mesh
    mesh = rect_with_hole_mesh(h=h)
    n = FacetNormal(mesh)

    # Normalise parameters
    h_c = Constant(h)
    degree_c = Constant(degree)
    sigma_c = Constant(sigma)
    nu_c = Constant(nu)
    dt_c = Constant(dt)

    # Spaces
    V = FunctionSpace(mesh, "BDM", degree)
    Q = FunctionSpace(mesh, "DG", degree-1)
    W = V * Q

    # Unknowns
    w = Function(W)
    u, p = split(w)
    u_sol = w.subfunctions

    # Old velocity
    u_old = Function(V)

    # Tests
    v, q = TestFunctions(W)


    # -----------------------
    #   Boundary conditions
    # -----------------------
    bcs = [
        DirichletBC(W.sub(0), Constant((1.0, 0.0)), 1),
        DirichletBC(W.sub(0), Constant((0.0, 0.0)), 3),
        DirichletBC(W.sub(0), Constant((0.0, 0.0)), 4),
    ]


    # ------------
    #   Residual
    # ------------
    # Operators
    dev = lambda u : sym(grad(u))
    jump = lambda u : outer(u, n)('+') + outer(u, n)('-')

    # Advective term
    advective_bulk = inner(dot(grad(u), u), v) * dx
    advective_int  = - inner(dot(jump(u), u), v) * dS

    # Viscous term
    viscous_bulk = inner(dev(u), dev(v)) * dx
    viscous_int = (
      - inner(avg(dev(u)), sym(jump(v)))
      - inner(sym(jump(u)), avg(dev(v)))
      + (sigma_c * degree**2 / h_c) * inner(sym(jump(u)), sym(jump(v)))
    ) * dS

    # Total Form
    F = (
        inner((u - u_old) / dt, v) * dx
      + advective_bulk + advective_int
      + 2 * nu_c * (viscous_bulk + viscous_int)
      - inner(p, div(v)) * dx
      - inner(q, div(u)) * dx
    )


    # -------------
    #   Time loop
    # -------------
    t = 0.0
    step = 0

    sp = {
        "snes_type": "newtonls",
        "snes_max_it": 20,
        "ksp_type": "gmres",
        "pc_type": "lu", # Direct solver for robustness
        "mat_type": "aij"
    }

    outfile = VTKFile("velocity.pvd")

    while t < T:
        t += float(dt)
        step += 1

        print(GREEN % f"Step {step} | t = {t:.4e}")

        solve(F == 0, w, bcs=bcs, solver_parameters=sp)

        outfile.write(u_sol)

        u_old.assign(u_sol)


    # ------------
    #   Download
    # ------------
    if download:
        import zipfile
        import glob
        import os
        from google.colab import files
        
        # Zip the .pvd and all .vtu files
        zip_name = "velocity_data.zip"
        with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
            if os.path.exists("velocity.pvd"):
                zipf.write("velocity.pvd")
            for filename in glob.glob("velocity*.vtu"):
                zipf.write(filename)
                
        files.download(zip_name)
        print(BLUE % f"Download complete!")

In [58]:
navier_stokes(download=True)

Step 1 | t = 3.1250e-02
Step 2 | t = 6.2500e-02
Step 3 | t = 9.3750e-02
Step 4 | t = 1.2500e-01


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>